In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [8]:
df = pd.read_csv('data/df_with_pca.csv')

In [9]:
x_vars = ["Indice di spopolamento","Indice di occupazione","Indice disoccupazione","Tipo di località_Altri comuni non altrimenti classificati","Tipo di località_Capoluogo senza specifici interessi turistici","Tipo di località_Città d\'Arte","Tipo di località_Località collinari","Tipo di località_Località lacuali","Tipo di località_Località marine","Tipo di località_Località montane","Tipo di località_Località religiose","Tipo di località_Località termali", "PC_Economica","PC_Turistica_1","PC_Turistica_2", "PC_Turistica_3", "PC_Turistica_4", "PC_Turistica_5", "PC_Turistica_6", "PC_Turistica_7", "PC_Turistica_8", "PC_Turistica_9", "PC_Turistica_10", "PC_Turistica_11", "PC_Turistica_12", "Macroarea_Isole", "Macroarea_Nord", "Macroarea_Sud","Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico", "Descrizione_Tipologia_di_Immobile_Abitazioni_signorili", "Descrizione_Tipologia_di_Immobile_Abitazioni_tipiche_dei_luoghi","Descrizione_Tipologia_di_Immobile_Ville_e_Villini"]

In [10]:
x = df[x_vars]
y= df['Costo Minimo alla Vendita']

KeyError: "['Macroarea_Isole', 'Macroarea_Nord', 'Macroarea_Sud', 'Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico', 'Descrizione_Tipologia_di_Immobile_Abitazioni_signorili', 'Descrizione_Tipologia_di_Immobile_Abitazioni_tipiche_dei_luoghi', 'Descrizione_Tipologia_di_Immobile_Ville_e_Villini'] not in index"